In [74]:
import os
import json
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from typing import List, Dict, Any, Tuple, Optional
from pinecone import Pinecone, ServerlessSpec
import logging
from dataclasses import dataclass, asdict
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional
import torch
import numpy as np
import cv2
from PIL import Image
import io
import pymupdf as fitz
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from llama_index.core import Document, Settings, StorageContext, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.llms.openai import OpenAI
from llama_parse import LlamaParse
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.auto import partition
import logging
from llama_index.core import Settings, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.readers.file import PDFReader
from datetime import datetime
from llama_index.core import Document, Settings, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings, VectorStoreIndex
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI
from llama_parse import LlamaParse
from typing import Sequence
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.auto import partition
from dataclasses import dataclass
from typing import List, Dict, Any
import pymupdf as fitz
import llama_index.core
os.environ["OPENAI_API_KEY"] = "xxxxxxx"
pc = Pinecone(api_key="xxxxxxx")



llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)
embed_model = HuggingFaceEmbedding(
            model_name="BAAI/bge-small-en-v1.5",
            embed_batch_size=10
        )
Settings.embed_model = embed_model
Settings.dimension = 384

Settings.llm = OpenAI(model="gpt-4o-mini", api_base="https://models.inference.ai.azure.com")
def optimize_metadata(metadata: dict) -> dict:
    """
    Convert complex metadata types to simple types that Pinecone accepts.
    Pinecone only accepts strings, numbers, booleans, or lists of strings.
    """
    optimized = {}
    
    # Handle essential fields
    essential_fields = ['source', 'type', 'page_num']
    for field in essential_fields:
        if field in metadata:
            optimized[field] = str(metadata[field])
    
    # Handle bbox - convert to string representation
    if 'bbox' in metadata:
        bbox = metadata['bbox']
        optimized['bbox_str'] = f"{bbox['x1']},{bbox['y1']},{bbox['x2']},{bbox['y2']}"
    
    # Handle paths - store just the filenames
    if 'image' in metadata:
        optimized['image_filename'] = os.path.basename(metadata['image'])
    if 'dataframe' in metadata:
        optimized['excel_filename'] = os.path.basename(metadata['dataframe'])
    
    # Handle captions
    if 'caption' in metadata:
        optimized['caption'] = str(metadata['caption'])[:512]  # Limit caption length
    
    return optimized

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('document_processing.log'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)
import os
import json
import logging
from dataclasses import dataclass, asdict
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional
import torch
import numpy as np
import cv2
from PIL import Image
import io
import pymupdf as fitz
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from llama_index.core import Document, Settings, StorageContext, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.llms.openai import OpenAI
from llama_parse import LlamaParse
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.auto import partition
def optimize_metadata(metadata: dict) -> dict:
    """
    Convert complex metadata types to simple types that Pinecone accepts.
    Pinecone only accepts strings, numbers, booleans, or lists of strings.
    """
    optimized = {}
    
    # Handle essential fields
    essential_fields = ['source', 'type', 'page_num']
    for field in essential_fields:
        if field in metadata:
            optimized[field] = str(metadata[field])
    
    # Handle bbox - convert to string representation
    if 'bbox' in metadata:
        bbox = metadata['bbox']
        optimized['bbox_str'] = f"{bbox['x1']},{bbox['y1']},{bbox['x2']},{bbox['y2']}"
    
    # Handle paths - store just the filenames
    if 'image' in metadata:
        optimized['image_filename'] = os.path.basename(metadata['image'])
    if 'dataframe' in metadata:
        optimized['excel_filename'] = os.path.basename(metadata['dataframe'])
    
    # Handle captions
    if 'caption' in metadata:
        optimized['caption'] = str(metadata['caption'])[:512]  # Limit caption length
    
    return optimized

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('document_processing.log'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

@dataclass
class ProcessingMetrics:
    start_time: datetime
    end_time: Optional[datetime] = None
    document_count: int = 0
    total_tokens: int = 0
    total_characters: int = 0
    processed_files: List[str] = None
    errors: List[Dict] = None
    
    def to_dict(self):
        return asdict(self)
    
    def log_error(self, filename: str, error: Exception):
        if self.errors is None:
            self.errors = []
        self.errors.append({
            "filename": filename,
            "error": str(error),
            "timestamp": datetime.now().isoformat()
        })

@dataclass
class DocumentElement:
    content: Any
    element_type: str
    page_num: int
    bbox: str
    source_file: str
    element_id: str
    citations: List[str] = None
    
    def to_document(self) -> Document:
        metadata = {
            "type": self.element_type,
            "page_num": self.page_num,
            "bbox": self.bbox,
            "source": self.source_file,
            "element_id": self.element_id,
            "citations": self.citations or []
        }
        
        if self.element_type == "table":
            metadata["table_path"] = self.content["table_path"]
            text = f"Table with caption: {self.content['caption']}\nColumns: {self.content['columns']}"
        elif self.element_type == "image":
            metadata["image_path"] = self.content["image_path"]
            text = f"Image with caption: {self.content['caption']}"
        else:
            text = self.content
            
        return Document(text=text, metadata=metadata, id_=self.element_id)

class DocumentProcessor:
    def __init__(self):
        self.metrics = ProcessingMetrics(
            start_time=datetime.now(),
            processed_files=[],
            errors=[]
        )
        
        # Initialize models
        self.vision_model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
        self.feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
        self.tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.vision_model.to(self.device)
        
        logger.info(f"Initialized DocumentProcessor with device: {self.device}")

    def process_text_block(self, text_block: tuple, page_num: int, source_file: str, block_id: int) -> DocumentElement:
        """Process a text block from PDF."""
        logger.debug(f"Processing text block {block_id} from page {page_num}")
        
        bbox = {
            "x1": text_block[0], 
            "y1": text_block[1],
            "x2": text_block[2], 
            "y2": text_block[3]
        }
        bbox = f"{bbox['x1']},{bbox['y1']},{bbox['x2']},{bbox['y2']}"
        
        self.metrics.total_characters += len(text_block[4])
        # Approximate token count
        self.metrics.total_tokens += len(text_block[4].split())
        
        return DocumentElement(
            content=text_block[4],
            element_type="text",
            page_num=page_num,
            bbox=bbox,
            source_file=source_file,
            element_id=f"{source_file}-page{page_num}-block{block_id}"
        )

    def process_table(self, table, page_num: int, source_file: str, table_id: int, 
                     save_dir: str) -> DocumentElement:
        """Process a table from PDF."""
        logger.debug(f"Processing table {table_id} from page {page_num}")
        
        df = table.to_pandas()
        table_path = os.path.join(save_dir, f"table-{table_id}-page-{page_num}.xlsx")
        df.to_excel(table_path, index=False)
        
        content = {
            "table_path": table_path,
            "caption": f"Table extracted from page {page_num}",
            "columns": ", ".join(df.columns.tolist())
        }
        
        # Update metrics
        self.metrics.total_characters += sum(df.astype(str).sum().sum())
        self.metrics.total_tokens += len(df.columns) * df.shape[0]  # Approximate
        
        return DocumentElement(
            content=content,
            element_type="table",
            page_num=page_num,
            bbox=f"{table.bbox['x1']},{table.bbox['y1']},{table.bbox['x2']},{table.bbox['y2']}",
            source_file=source_file,
            element_id=f"{source_file}-page{page_num}-table{table_id}"
        )

    def describe_image(self, image_content: bytes) -> str:
        """Generate a text description of the image using a pre-trained model."""
        logger.debug("Generating image description")
        
        image = Image.open(io.BytesIO(image_content)).convert('RGB')
        pixel_values = self.feature_extractor(images=[image], return_tensors="pt").pixel_values
        pixel_values = pixel_values.to(self.device)

        output_ids = self.vision_model.generate(
            pixel_values,
            max_length=16,
            num_beams=4
        )

        preds = self.tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        description = preds[0].strip()
        
        # Update metrics
        self.metrics.total_characters += len(description)
        self.metrics.total_tokens += len(description.split())
        
        return description

    def is_graph(self, image_data: bytes) -> bool:
        """Detect if the image contains a graph."""
        logger.debug("Checking if image is a graph")
        
        nparr = np.frombuffer(image_data, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150, apertureSize=3)
        lines = cv2.HoughLines(edges, 1, np.pi/180, threshold=100)
        
        return lines is not None and len(lines) > 10

    def process_graph(self, image_data: bytes) -> str:
        """Process a graph image to extract basic information."""
        logger.debug("Processing graph")
        
        nparr = np.frombuffer(image_data, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150, apertureSize=3)
        lines = cv2.HoughLines(edges, 1, np.pi/180, threshold=100)
        
        horizontal_lines = 0
        vertical_lines = 0
        
        if lines is not None:
            for line in lines:
                rho, theta = line[0]
                if np.abs(theta) < np.pi / 4 or np.abs(theta - np.pi) < np.pi / 4:
                    vertical_lines += 1
                else:
                    horizontal_lines += 1
        
        graph_type = "bar graph" if vertical_lines > horizontal_lines else \
                    "line graph" if horizontal_lines > vertical_lines else \
                    "scatter plot"
        
        description = f"This image appears to be a {graph_type} with approximately {vertical_lines} vertical lines and {horizontal_lines} horizontal lines."
        
        # Update metrics
        self.metrics.total_characters += len(description)
        self.metrics.total_tokens += len(description.split())
        
        return description

    def parse_all_tables(self, filename: str, page, pagenum: int, text_blocks, ongoing_tables) -> Tuple[List[Document], List, Any]:
        """Extract tables from a PDF page."""
        logger.debug(f"Parsing tables from page {pagenum}")
        
        table_docs = []
        table_bboxes = []
        
        try:
            tables = page.find_tables(
                horizontal_strategy="lines_strict",
                vertical_strategy="lines_strict"
            )
            
            tablerefdir = os.path.join(os.getcwd(), "vectorstore/table_references")
            os.makedirs(tablerefdir, exist_ok=True)
            
            for tab in tables:
                if not tab.header.external:
                    continue
                
                pandas_df = tab.to_pandas()
                df_xlsx_path = os.path.join(
                    tablerefdir,
                    f"table{len(table_docs)+1}-page{pagenum}.xlsx"
                )
                pandas_df.to_excel(df_xlsx_path)
                
                bbox = fitz.Rect(tab.bbox)
                table_bboxes.append(bbox)
                
                # Extract surrounding text
                before_text, after_text = self.extract_text_around_item(
                    text_blocks,
                    bbox,
                    page.rect.height
                )
                
                # Save table image
                table_img = page.get_pixmap(clip=bbox)
                table_img_path = os.path.join(
                    tablerefdir,
                    f"table{len(table_docs)+1}-page{pagenum}.jpg"
                )
                table_img.save(table_img_path)
                
                description = self.process_graph(table_img.tobytes())
                caption = before_text.replace("\n", " ") + description + after_text.replace("\n", " ")
                if before_text == "" and after_text == "":
                    caption = " ".join(tab.header.names)
                
                table_metadata = {
                    "source": f"{filename[:-4]}-page{pagenum}-table{len(table_docs)+1}",
                    "excel_path": df_xlsx_path,
                    "image_path": table_img_path,
                    "caption": caption,
                    "type": "table",
                    "page_num": pagenum,
                    "columns": list(pandas_df.columns.values),
                    "markdown_table": pandas_df.to_markdown(),
                    "html_table": pandas_df.to_html()
                }
                
                doc_text = (
                    f"Table Reference: {table_metadata['markdown_table']}\n"
                    f"Caption: {caption}\n"
                    f"Excel File: {df_xlsx_path}\n"
                    f"Table Image: ![Table]({table_img_path})"
                )
                optimized_metadata = optimize_metadata(table_metadata)
                table_docs.append(Document(
                    text=doc_text,
                    metadata=optimized_metadata
                ))
                
                # Update metrics
                self.metrics.total_characters += len(doc_text)
                self.metrics.total_tokens += len(doc_text.split())
                
        except Exception as e:
            logger.error(f"Error processing tables on page {pagenum}: {e}")
            self.metrics.log_error(filename, e)
        
        return table_docs, table_bboxes, ongoing_tables

    def load_data_from_directory(self, directory: str) -> List[Document]:
        """Load and process multiple file types from a directory."""
        logger.info(f"Processing directory: {directory}")
        
        documents = []
        
        for filename in os.listdir(directory):
            filepath = os.path.join(directory, filename)
            file_extension = os.path.splitext(filename.lower())[1]
            
            logger.info(f"Processing file: {filename}")
            self.metrics.processed_files.append(filename)
            
            try:
                if file_extension in ('.png', '.jpg', '.jpeg'):
                    with open(filepath, "rb") as image_file:
                        image_content = image_file.read()
                        image_text = self.describe_image(image_content)
                        doc = Document(
                            text=image_text,
                            metadata={"source": filename, "type": "image"}
                        )
                        documents.append(doc)
                
                elif file_extension == '.pdf':
                    with open(filepath, "rb") as pdf_file:
                        pdf_documents = self.get_pdf_documents(pdf_file)
                        documents.extend(pdf_documents)
                
                elif file_extension in ('.ppt', '.pptx'):
                    ppt_documents = self.process_ppt_file(filepath)
                    documents.extend(ppt_documents)
                
                else:
                    with open(filepath, "r", encoding="utf-8") as text_file:
                        text = text_file.read()
                        doc = Document(
                            text=text,
                            metadata={"source": filename, "type": "text"}
                        )
                        documents.append(doc)
                
                self.metrics.document_count += 1
                
            except Exception as e:
                logger.error(f"Error processing {filename}: {e}")
                self.metrics.log_error(filename, e)
                continue
        
        return documents

    def export_metrics(self, output_path: str = "processing_metrics.json"):
        """Export processing metrics to JSON file."""
        self.metrics.end_time = datetime.now()
        
        with open(output_path, 'w') as f:
            json.dump(self.metrics.to_dict(), f, indent=2, default=str)
        
        logger.info(f"Metrics exported to {output_path}")
        
        return self.metrics.to_dict()

    # Helper method that needs to be implemented
    # ... [Previous code remains the same until extract_text_around_item] ...

    def extract_text_around_item(self, text_blocks, bbox, page_height, context_distance: float = 50.0) -> Tuple[str, str]:
        """
        Extract text before and after an item on the page within a specified distance.
        
        Args:
            text_blocks: List of text blocks from the page
            bbox: Bounding box of the item (table/image)
            page_height: Height of the page
            context_distance: Distance in points to look for context (default: 50.0)
            
        Returns:
            Tuple[str, str]: (text before item, text after item)
        """
        logger.debug("Extracting text around item")
        
        # Convert bbox to coordinates if it's not already
        if isinstance(bbox, dict):
            item_top = bbox['y1']
            item_bottom = bbox['y2']
        else:
            item_top = bbox.y0
            item_bottom = bbox.y1
            
        before_text = []
        after_text = []
        
        for block in text_blocks:
            # Get the vertical position of the text block
            block_bottom = block[3]  # y2
            block_top = block[1]     # y1
            block_text = block[4]    # text content
            
            # Check if text is before the item
            if block_bottom < item_top and (item_top - block_bottom) <= context_distance:
                before_text.append(block_text)
                
            # Check if text is after the item
            elif block_top > item_bottom and (block_top - item_bottom) <= context_distance:
                after_text.append(block_text)
        
        return " ".join(before_text), " ".join(after_text)

    def get_pdf_documents(self, pdf_file) -> List[Document]:
        """
        Process PDF file and return list of documents.
        
        Args:
            pdf_file: File object of the PDF
            
        Returns:
            List[Document]: List of processed documents
        """
        logger.info("Processing PDF file")
        
        try:
            # Load PDF
            pdf_document = fitz.open(stream=pdf_file.read())
            documents = []
            
            for page_num in range(len(pdf_document)):
                page = pdf_document[page_num]
                
                # Get text blocks
                text_blocks = page.get_text("blocks")
                
                # Process tables first
                table_docs, table_bboxes, _ = self.parse_all_tables(
                    pdf_file.name,
                    page,
                    page_num + 1,
                    text_blocks,
                    None
                )
                documents.extend(table_docs)
                
                # Process remaining text blocks
                for block_id, block in enumerate(text_blocks):
                    # Skip if block overlaps with any table
                    block_rect = fitz.Rect(block[:4])
                    skip_block = any(block_rect.intersects(table_bbox) for table_bbox in table_bboxes)
                    
                    if not skip_block:
                        doc_element = self.process_text_block(
                            block,
                            page_num + 1,
                            pdf_file.name,
                            block_id
                        )
                        documents.append(doc_element.to_document())
                
                # Process images
                images = page.get_images(full=True)
                for img_id, img_info in enumerate(images):
                    xref = img_info[0]
                    base_image = pdf_document.extract_image(xref)
                    
                    if base_image:
                        image_data = base_image["image"]
                        
                        # Process as graph if detected
                        if self.is_graph(image_data):
                            description = self.process_graph(image_data)
                        else:
                            description = self.describe_image(image_data)
                        
                        # Create image document
                        doc = Document(
                            text=description,
                            metadata={
                                "source": pdf_file.name,
                                "type": "image",
                                "page_num": page_num + 1,
                                "image_id": img_id
                            }
                        )
                        documents.append(doc)
            
            return documents
            
        except Exception as e:
            logger.error(f"Error processing PDF: {e}")
            self.metrics.log_error(pdf_file.name, e)
            return []

    def process_ppt_file(self, filepath: str) -> List[Document]:
        """
        Process PowerPoint file and return list of documents.
        
        Args:
            filepath: Path to the PowerPoint file
            
        Returns:
            List[Document]: List of processed documents
        """
        logger.info(f"Processing PowerPoint file: {filepath}")
        
        try:
            from pptx import Presentation
            
            documents = []
            prs = Presentation(filepath)
            
            for slide_num, slide in enumerate(prs.slides, 1):
                # Process text content
                text_content = []
                
                for shape in slide.shapes:
                    if hasattr(shape, "text") and shape.text.strip():
                        text_content.append(shape.text.strip())
                        
                        # Update metrics
                        self.metrics.total_characters += len(shape.text)
                        self.metrics.total_tokens += len(shape.text.split())
                
                if text_content:
                    doc = Document(
                        text="\n".join(text_content),
                        metadata={
                            "source": filepath,
                            "type": "presentation",
                            "slide_num": slide_num
                        }
                    )
                    documents.append(doc)
                
                # Process images in the slide
                for shape in slide.shapes:
                    if hasattr(shape, "image"):
                        try:
                            # Save image temporarily
                            image_bytes = shape.image.blob
                            
                            # Process as graph if detected
                            if self.is_graph(image_bytes):
                                description = self.process_graph(image_bytes)
                            else:
                                description = self.describe_image(image_bytes)
                            
                            doc = Document(
                                text=description,
                                metadata={
                                    "source": filepath,
                                    "type": "presentation_image",
                                    "slide_num": slide_num
                                }
                            )
                            documents.append(doc)
                            
                        except Exception as e:
                            logger.warning(f"Error processing image in slide {slide_num}: {e}")
                            continue
            
            return documents
            
        except Exception as e:
            logger.error(f"Error processing PowerPoint file: {e}")
            self.metrics.log_error(filepath, e)
            return []

2024-11-03 22:33:40,320 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/Users/saisuryamadhav/Library/Caches/pypoetry/virtualenvs/backend-dSPC7ywe-py3.12/lib/python3.12/site-packages/pinecone_plugins'])
2024-11-03 22:33:40,324 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2024-11-03 22:33:40,324 - pinecone_plugin_interface.logging - INFO - Installing plugin inference into Pinecone
2024-11-03 22:33:40,664 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2024-11-03 22:33:46,132 - sentence_transformers.SentenceTransformer - INFO - 2 prompts are loaded, with the keys: ['query', 'text']


In [58]:
import os
from pathlib import Path
from typing import List, Optional
from llama_index.core import Settings, StorageContext, VectorStoreIndex
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from pinecone import Pinecone, ServerlessSpec

def initialize_vector_store(index_name: str = "document-store") -> PineconeVectorStore:
    """Initialize Pinecone vector store with the new client."""
    
    # Initialize Pinecone client
    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
    
    # Check if index exists, if not create it
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=Settings.dimension,  # OpenAI embedding dimension
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )
    
    # Initialize vector store
    vector_store = PineconeVectorStore(
        index_name=index_name,
        environment=os.environ.get("PINECONE_ENVIRONMENT")
    )
    
    return vector_store

In [59]:
def create_query_engine(index: VectorStoreIndex):
    """Create a query engine with specific configurations."""
    return index.as_query_engine(
        similarity_top_k=5,
        response_mode="tree_summarize"
    )

def search_documents(query: str, index: VectorStoreIndex, 
                    filters: Dict = None) -> Dict:
    """Search documents with optional filters."""
    query_engine = create_query_engine(index)
    response = query_engine.query(query)
    
    # Extract source nodes and metadata
    sources = []
    for node in response.source_nodes:
        sources.append({
            "text": node.text,
            "metadata": node.metadata,
            "score": node.score
        })
    
    return {
        "answer": response.response,
        "sources": sources
    }

In [60]:
config = {
    "input_directory": "/Users/saisuryamadhav/Documents/University/new/Assignment_3_Team1/backend/pdf",
    "pinecone_api_key": "xxxxxxx",
    "pinecone_environment": "us-east1-gcp",
    "openai_api_key": "xxxxxxx",
    "index_name": "document-store"
}

In [61]:
def main(
    input_directory: str,
    pinecone_api_key: str,
    pinecone_environment: str,
    openai_api_key: str,
    index_name: str = "document-store",
    batch_size: int = 100
) -> Optional[VectorStoreIndex]:
    """
    Main function to process documents and create searchable index.
    
    Args:
        input_directory: Path to directory containing PDFs
        pinecone_api_key: Pinecone API key
        pinecone_environment: Pinecone environment
        openai_api_key: OpenAI API key
        index_name: Name of the Pinecone index
        batch_size: Number of documents to process in each batch
    """
    try:
        # Set up environment variables
        os.environ["OPENAI_API_KEY"] = openai_api_key
        os.environ["PINECONE_API_KEY"] = pinecone_api_key
        os.environ["PINECONE_ENVIRONMENT"] = pinecone_environment

        
        from llama_index.embeddings.huggingface import HuggingFaceEmbedding

        # Initialize the embedding model        
        # Initialize Pinecone
        vector_store = initialize_vector_store()

        # Create storage context
        storage_context = StorageContext.from_defaults(
            vector_store=vector_store
        )

        # Process documents
        print(f"Processing documents from {input_directory}")
        processor = DocumentProcessor()
        documents = processor.load_data_from_directory(input_directory)
        
        if not documents:
            print("No documents were processed successfully.")
            return None

        print(f"Successfully processed {len(documents)} documents")
        
        # Create index
        print("Creating vector index...")
        index = VectorStoreIndex.from_documents(
            documents,
            storage_context=storage_context,
            show_progress=True
        )
        
        print("Index created successfully!")
        
        
        return index

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None


In [62]:
# Run the main processing
Settings.chunk_size = 1024 
Settings.chunk_overlap = 20
Settings.metadata_max_length = 1000  # Add metadata length limit
index = main(**config)

2024-11-03 19:22:40,014 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/Users/saisuryamadhav/Library/Caches/pypoetry/virtualenvs/backend-dSPC7ywe-py3.12/lib/python3.12/site-packages/pinecone_plugins'])
2024-11-03 19:22:40,016 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2024-11-03 19:22:40,016 - pinecone_plugin_interface.logging - INFO - Installing plugin inference into Pinecone
2024-11-03 19:22:40,214 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/Users/saisuryamadhav/Library/Caches/pypoetry/virtualenvs/backend-dSPC7ywe-py3.12/lib/python3.12/site-packages/pinecone_plugins'])
2024-11-03 19:22:40,215 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2024-11-03 19:22:40,215 - pinecone_plugin_interface.logging - INFO - Installing plugin inference into Pinecone


Processing documents from /Users/saisuryamadhav/Documents/University/new/Assignment_3_Team1/backend/pdf


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.46.1"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "activation_function": "gelu_new",
  "add_cross_attention": true,
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_rang

Successfully processed 1567 documents
Creating vector index...


Parsing nodes:   0%|          | 0/1567 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/1567 [00:00<?, ?it/s]

Index created successfully!


In [63]:
def create_query_engine(index: VectorStoreIndex):
    """Create a query engine with specific configurations."""
    return index.as_query_engine(
        similarity_top_k=5,
        response_mode="tree_summarize"
    )

def search_documents(query: str, index: VectorStoreIndex, 
                    filters: Dict = None) -> Dict:
    """Search documents with optional filters."""
    query_engine = create_query_engine(index)
    response = query_engine.query(query)
    
    # Extract source nodes and metadata
    sources = []
    for node in response.source_nodes:
        sources.append({
            "text": node.text,
            "metadata": node.metadata,
            "score": node.score
        })
    
    return {
        "answer": response.response,
        "sources": sources
    }

In [64]:
query_engine = index.as_query_engine(
            similarity_top_k=5,
            response_mode="tree_summarize"
        )
        
print("\nTesting index with a sample query...")
response = query_engine.query(
    "What are the main topics covered in these documents?"
)
print("\nSample Query Response:")
print(response.response)



Testing index with a sample query...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-11-03 19:25:11,626 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"



Sample Query Response:
The main topics covered in the documents include consumer goods, infrastructure, and a global overview of various asset classes across different domiciles. Additionally, there is a statement of purpose that likely outlines the objectives or intentions of the content.


In [70]:
def format_query_response(response, query_type="general"):
    """Format query response based on content type."""
    
    if query_type == "images":
        formatted_response = "Here are the relevant images:\n\n"
        for node in response.source_nodes:
            if node.metadata.get("type") == "image":
                formatted_response += (
                    f"## Image on page {node.metadata['page_num']}\n"
                    f"{node.metadata['markdown_link']}\n"
                    f"**Caption:** {node.metadata['caption']}\n\n"
                )
    
    elif query_type == "tables":
        formatted_response = "Here are the relevant tables:\n\n"
        for node in response.source_nodes:
            if node.metadata.get("type") == "table":
                formatted_response += (
                    f"## Table from {node.metadata['source']}\n"
                    f"{node.metadata['markdown_table']}\n"
                    f"**Caption:** {node.metadata['caption']}\n"
                    f"[Download Excel]({node.metadata['excel_path']})\n\n"
                )
    
    else:
        formatted_response = response.response
    print(formatted_response)
    return formatted_response

def query_documents(query: str, index: VectorStoreIndex):
    query_engine = index.as_query_engine(
        similarity_top_k=5,
        response_mode="tree_summarize"
    )
    
    # Determine query type
    query_type = "general"
    if "image" in query.lower():
        query_type = "images"
    elif "table" in query.lower():
        query_type = "tables"
    
    # Get and format response
    response = query_engine.query(query)
    formatted_response = format_query_response(response, query_type)
    
    return formatted_response
if index:
    # Example queries
    queries = [
        "What are the main topics covered?",
        "Show me any tables related to financial data",
        "What images are present in the documents?"
    ]
    
    for query in queries:
        print(f"\nQuery: {query}")
        response = query_engine.query(query)
        print("Response:", response)


Query: What are the main topics covered?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-11-03 19:29:35,927 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


Response: The main topics covered include the current state of the active versus passive debate, the beginning of this debate, and a detailed analysis of the evolution of active and passive investing. Additionally, there is a framing of the active versus passive debate and implications for the future of asset management.

Query: Show me any tables related to financial data


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-11-03 19:29:37,356 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


Response: There are no specific tables mentioned in the provided information related to financial data. However, there are references to granular line-item details and classifications, such as the US Equity Large-Cap Blend's percentage of total at the end of 1999, which may imply the presence of detailed financial data in the document. For specific tables, further examination of the document would be necessary.

Query: What images are present in the documents?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-11-03 19:29:37,984 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


Response: The document contains an image that appears to be a line graph with approximately 2246 vertical lines and 2270 horizontal lines.
